In [3]:
import pandas as pd
import random
from datetime import datetime

# Definir advogados e gerar IDs unicos
advogados = [
    "Ana Cláudia Martins", "Bruno Ferreira da Silva", "Camila Souza Lopes", "Daniel Oliveira Costa",
    "Eduardo Almeida Ramos", "Fernanda Lima Pereira", "Gabriela Santos Rocha", "Henrique Barbosa de Souza",
    "Isabela Mendes Oliveira", "João Paulo Cardoso"
]

# Gerar DataFrame de advogados com atributos
anos_contratacao = [random.randint(2005, 2022) for _ in advogados]
datas_contratacao = [datetime(year=ano, month=random.randint(1, 12), day=random.randint(1, 28)) for ano in anos_contratacao]
tempo_servico = [datetime.now().year - ano for ano in anos_contratacao]
grau_formacao = [random.choices(["Graduado", "Pós-Graduado", "Mestre", "Doutor"], weights=[50, 30, 15, 5])[0] for _ in advogados]
genero = [random.choice(["Masculino", "Feminino", "Outro"]) for _ in advogados]
unidades = [random.choices(["Santa Maria", "Porto Alegre", "Uruguaiana"], weights=[40, 40, 20])[0] for _ in advogados]

# Criar DataFrame dos advogados
df_advogados = pd.DataFrame({
    'ID_advogado': range(1, len(advogados) + 1),
    'nome': advogados,
    'ano_contratacao': anos_contratacao,
    'data_contratacao': datas_contratacao,
    'tempo_servico': tempo_servico,
    'grau_formacao': grau_formacao,
    'genero': genero,
    'unidade': unidades
})

# Pesos para procedência baseado na formação
grau_procedencia_pesos = {
    "Graduado": [30, 50, 20],
    "Pós-Graduado": [40, 50, 10],
    "Mestre": [50, 40, 10],
    "Doutor": [60, 30, 10]
}

# Pesos para anos
anos = list(range(2005, 2025))
pesos_anos = [1] * 11 + [3] * 5 + [17] * 2 + [8] * 2 + [2] * 2  # Mais ênfase em 2021 e 2022

# Assuntos com faixas de valores e pesos de frequência
assuntos = {
    "Aposentadoria por Idade Urbana": (10000, 30000, 4),
    "Aposentadoria por Tempo de Contribuição": (15000, 50000, 2),
    "Aposentadoria por Invalidez": (20000, 60000, 2),
    "Auxílio-Doença": (5000, 20000, 10),
    "Auxílio-Acidente": (5000, 20000, 8),
    "Benefício Assistencial (LOAS/BPC)": (3000, 15000, 6),
    "Revisão de Aposentadoria": (25000, 70000, 1),
    "Pensão por Morte": (15000, 40000, 3),
    "Salário-Maternidade": (2000, 10000, 8),
    "Reconhecimento de Tempo Especial": (20000, 60000, 1),
    "Revisão da Vida Toda": (30000, 80000, 1),
    "Restabelecimento de Benefício": (10000, 30000, 3),
    "Concessão de Benefício Previdenciário": (8000, 25000, 5),
    "Revisão de Benefício Previdenciário": (15000, 50000, 2)
}

procedencias = ["Procedente", "Parcial Procedência", "Improcedente"]
cidades = [
    "Santa Maria", "Porto Alegre", "Uruguaiana", "Pelotas", "Caxias do Sul", "Passo Fundo",
    "Novo Hamburgo", "Santa Cruz do Sul", "São Leopoldo", "Bagé", "Santo Ângelo", "Rio Grande",
    "Lajeado", "Ijuí", "Canoas", "Erechim"
]
pesos_cidades = [30, 30, 15, 5, 4, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]

# Função para gerar número do processo CNJ fictício
def gerar_num_processo():
    sequencial = str(random.randint(1, 9999999)).zfill(7)
    dv = str(random.randint(0, 99)).zfill(2)
    ano = str(random.randint(2005, 2024))
    segmento = '1'
    tribunal = '04'
    origem = str(random.randint(1, 999)).zfill(4)
    return f"{sequencial}-{dv}.{ano}.{segmento}.{tribunal}.{origem}", int(ano)

# Gerar dados dos processos
dados = []
num_registros = 4832

for _ in range(num_registros):
    num_processo, ano_ajuizamento = gerar_num_processo()
    advogado = df_advogados.sample(weights='tempo_servico').iloc[0]
    mes = random.choices([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], weights=[5, 5, 5, 5, 5, 5, 25, 5, 5, 5, 20, 20], k=1)[0]
    dia = random.randint(1, 28)
    data_ajuizamento = datetime(ano_ajuizamento, mes, dia)
    cidade = random.choices(cidades, weights=pesos_cidades, k=1)[0]
    assunto, (min_valor, max_valor, peso) = random.choices(list(assuntos.items()), weights=[v[2] for v in assuntos.values()])[0]
    valor_causa = round(random.uniform(min_valor, max_valor), 2)
    procedencia = random.choices(procedencias, weights=grau_procedencia_pesos[advogado['grau_formacao']], k=1)[0]

    honor_contrat = round(valor_causa * 0.2, 2) if procedencia != "Improcedente" else 0.0
    honor_sucum = round(valor_causa * random.uniform(0.1, 0.15), 2) if procedencia != "Improcedente" else 0.0

    valor_devido = valor_causa + honor_contrat + honor_sucum if procedencia == "Procedente" else (valor_causa / 2 + honor_contrat + honor_sucum if procedencia == "Parcial Procedência" else 0.0)

    dados.append({
        "num_processo": num_processo,
        "ID_advogado": advogado['ID_advogado'],
        "ano_ajuizamento": ano_ajuizamento,
        "data_ajuizamento": data_ajuizamento,
        "cidade": cidade,
        "assunto": assunto,
        "valor_causa": valor_causa,
        "honor_contrat": honor_contrat,
        "honor_sucum": honor_sucum,
        "procedencia": procedencia,
        "valor_devido": round(valor_devido, 2)
    })

# Criar DataFrame dos processos
df_processos = pd.DataFrame(dados)

In [4]:
df_advogados.to_excel('dados_advogados.xlsx')
df_processos.to_excel('dados_processos.xlsx')